In [1]:
#import requests
#import json
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import RouteBuilder
from IPython.core.display import display, HTML
%matplotlib inline

## Data

In [2]:
stores = pd.read_csv("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\ups_coords.txt", names=["lat", "lon"])
custs = pd.read_csv("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cust_coords.txt", names=["lat", "lon"])

In [4]:
stores.iloc[0]

lat    42.985778
lon   -78.869874
Name: 0, dtype: float64

In [15]:
len(stores)

18

In [17]:
len(custs)

63

## Plots

In [19]:
m = folium.Map(location=(42.93, -78.79), zoom_start=11, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)
fg = folium.FeatureGroup(name="stores", overlay=False, show=False)
for i in range(len(stores)):
    folium.CircleMarker(location=tuple(stores.iloc[i]), color="red", radius=4).add_to(fg)
fg.add_to(m)
fg1 = folium.FeatureGroup(name="stores", overlay=False, show=False)
for i in range(len(custs)):
    folium.CircleMarker(location=tuple(custs.iloc[i]), color="orange", radius=3.5).add_to(fg1)
fg1.add_to(m)
m

## Routing distance

In [22]:
rb = RouteBuilder.RouteBuilder()
ss_dists_lst = []
for i in range(len(stores)):
    ss_dists_lst.append([])
    for j in range(i):
        ss_dists_lst[i].append(rb.makeRequest1(stores.iloc[i], stores.iloc[j]))

In [29]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\ss_dists.txt", 'w') as f:
    for x in ss_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")

In [3]:
rb = RouteBuilder.RouteBuilder()
cc_dists_lst = []
for i in range(len(custs)):
    cc_dists_lst.append([])
    for j in range(i):
        cc_dists_lst[i].append(rb.makeRequest1(custs.iloc[i], custs.iloc[j]))

In [5]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_dists.txt", 'w') as f:
    for x in cc_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")

In [6]:
rb = RouteBuilder.RouteBuilder()
sc_dists_lst = []
for i in range(len(stores)):
    sc_dists_lst.append([])
    for j in range(len(custs)):
        sc_dists_lst[i].append(rb.makeRequest1(stores.iloc[i], custs.iloc[j]))

In [7]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_dists.txt", 'w') as f:
    for x in sc_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")